In [60]:
import pandas as pd
import os
import re

### Extract Features

In [61]:
column_names = ["text","description","action","expected_response","positive"]
data = pd.DataFrame(columns=column_names)

In [62]:
def parse_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    cases = text.split('\n\n')
    for i in range(len(cases)):
        cases[i] = cases[i][3:].replace("   - ", "").replace("*", "")
    df = pd.DataFrame(columns=column_names)
    for case in cases:
        lines = case.strip().split('\n')
        if len(lines) >= 3:
            desc = lines[0].strip()
            if len(desc) == 0:
                desc = None

            action_line = lines[1]
            action_split = action_line.split(':')
            if len(action_split) == 2:
                action = action_split[1].strip()
            else:
                action = None

            expected_response_line = lines[2]
            expected_response_split = expected_response_line.split(':')
            if len(expected_response_split) == 2:
                expected_response = expected_response_split[1].strip()
            else:
                expected_response = None

            if 'Positive' in file_path:
                positive = True
            else:
                positive = False

            df_row = pd.DataFrame([[case.lower(),desc.lower(),action.lower(),expected_response.lower(),positive]], columns=column_names)
            df = pd.concat([df, df_row], ignore_index=True)

    return df

In [63]:
raw_data_path = 'raw/'
txt_list = os.listdir(raw_data_path)
for txt in txt_list:
    txt_df = parse_text_file(raw_data_path + txt)
    data = pd.concat([data,txt_df], ignore_index=True)

In [64]:
data.head()

,text,description,action,expected_response,positive
0,администратор удаляет товар из каталога.\nдейс...,администратор удаляет товар из каталога.,"delete-запрос к конечной точке ""/catalog/remov...",подтверждение удаления товара из каталога.,True
1,модератор удаляет комментарий под постом.\nдей...,модератор удаляет комментарий под постом.,"delete-запрос к конечной точке ""/post/comment/...",подтверждение удаления комментария.,True
2,менеджер удаляет клиента из базы данных.\nдейс...,менеджер удаляет клиента из базы данных.,"delete-запрос к конечной точке ""/database/cust...",подтверждение удаления клиента из базы данных.,True
3,редактор удаляет статью из журнала.\nдействие:...,редактор удаляет статью из журнала.,"delete-запрос к конечной точке ""/magazine/arti...",подтверждение удаления статьи из журнала.,True
4,модератор удаляет изображение из галереи.\nдей...,модератор удаляет изображение из галереи.,"delete-запрос к конечной точке ""/gallery/image...",подтверждение удаления изображения из галереи.,True


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               400 non-null    object
 1   description        400 non-null    object
 2   action             400 non-null    object
 3   expected_response  400 non-null    object
 4   positive           400 non-null    object
dtypes: object(5)
memory usage: 15.8+ KB


In [66]:
data.describe()

,text,description,action,expected_response,positive
count,400,400,400,400,400
unique,387,380,337,364,2
top,"дизайнер пытается удалить файл, который был у...",покупатель удаляет товар из корзины перед офор...,"get-запрос на эндпоинт ""/realtor/properties/999"".",код ошибки 404 - товар не найден.,True
freq,2,2,4,3,200


In [67]:
def get_req_type(action):
    action_split = action.split('-')
    if len(action_split) >= 2:
        req = action_split[0].strip()
    else:
        req = None
    return req

In [68]:
def get_actor(action):
    dsc_split = action.split('ет')
    actor = None
    if len(dsc_split) >= 2:
        dsc_split_0 = dsc_split[0].strip()
        dsc_double_split = dsc_split_0.split(' ')
        if len(dsc_double_split) >= 2:
            space = ' '
            actor = space.join(dsc_double_split[:-1]).strip().lower()
        elif len(dsc_split) >= 3:
            dsc_split_01 = ''.join(dsc_split[:2])
            dsc_double_split = dsc_split_01.split(' ')
            if len(dsc_double_split) >= 2:
                space = ' '
                actor = space.join(dsc_double_split[:-1]).strip().lower()
    return actor

In [69]:
def get_response_code(expectation):
    numbers = re.findall(r'\d+', expectation)
    if len(numbers) >= 1:
        return numbers[0].strip()
    else:
        return '200'

In [70]:
def get_endpoint(action):
    action_split = action.split('"')
    if len(action_split) >= 2:
        endpoint = action_split[1].strip()
    else:
        endpoint = None
    return endpoint

In [71]:
data['actor'] = data['description'].apply(get_actor)
data['request_type'] = data['action'].apply(get_req_type)
data['endpoint'] = data['action'].apply(get_endpoint)
data['response_code'] = data['expected_response'].apply(get_response_code)

In [72]:
data.head()

,text,description,action,expected_response,positive,actor,request_type,endpoint,response_code
0,администратор удаляет товар из каталога.\nдейс...,администратор удаляет товар из каталога.,"delete-запрос к конечной точке ""/catalog/remov...",подтверждение удаления товара из каталога.,True,администратор,delete,/catalog/remove,200
1,модератор удаляет комментарий под постом.\nдей...,модератор удаляет комментарий под постом.,"delete-запрос к конечной точке ""/post/comment/...",подтверждение удаления комментария.,True,модератор,delete,/post/comment/remove,200
2,менеджер удаляет клиента из базы данных.\nдейс...,менеджер удаляет клиента из базы данных.,"delete-запрос к конечной точке ""/database/cust...",подтверждение удаления клиента из базы данных.,True,менеджер,delete,/database/customer/remove,200
3,редактор удаляет статью из журнала.\nдействие:...,редактор удаляет статью из журнала.,"delete-запрос к конечной точке ""/magazine/arti...",подтверждение удаления статьи из журнала.,True,редактор,delete,/magazine/article/remove,200
4,модератор удаляет изображение из галереи.\nдей...,модератор удаляет изображение из галереи.,"delete-запрос к конечной точке ""/gallery/image...",подтверждение удаления изображения из галереи.,True,модератор,delete,/gallery/image/remove,200


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               400 non-null    object
 1   description        400 non-null    object
 2   action             400 non-null    object
 3   expected_response  400 non-null    object
 4   positive           400 non-null    object
 5   actor              400 non-null    object
 6   request_type       400 non-null    object
 7   endpoint           400 non-null    object
 8   response_code      400 non-null    object
dtypes: object(9)
memory usage: 28.2+ KB


In [74]:
data.describe()

,text,description,action,expected_response,positive,actor,request_type,endpoint,response_code
count,400,400,400,400,400,400,400,400,400
unique,387,380,337,364,2,59,4,272,20
top,"дизайнер пытается удалить файл, который был у...",покупатель удаляет товар из корзины перед офор...,"get-запрос на эндпоинт ""/realtor/properties/999"".",код ошибки 404 - товар не найден.,True,администратор,delete,/cart/item/remove,200
freq,2,2,4,3,200,44,100,10,200


In [75]:
data.to_csv("synthetic_testcases.csv", sep=';')
data.to_excel("synthetic_testcases.xlsx")

In [76]:
data.loc[383]

text                  продавец пытается изменить цену товара на "na...
description           продавец пытается изменить цену товара на "nan".
action               put-запрос на эндпоинт "/seller/products/{prod...
expected_response        код ошибки 400 - неверный формат цены товара.
positive                                                         False
actor                                                         продавец
request_type                                                       put
endpoint                           /seller/products/{product_id}/price
response_code                                                      400
Name: 383, dtype: object

### Label for Named Entity Recognition (NER)

In [77]:
ner_column_names = ['testcase_id', 'word_num', 'word', 'label']
labels = ['actor', 'request_type', 'endpoint', 'response_code']

In [78]:
def split_sentence(s):
    pattern = r'"(?:\\"|[^"])*"|\b\w+\b|[".,!?:-]'
    return re.findall(pattern, s)

In [79]:
def split_quotes(word_seq):
    for i in range(len(word_seq)):
        w = word_seq[i]
        if len(w) > 1 and '\"' in w:
            w = w[1:-1]
            word_seq[i] = '\"'
            word_seq.insert(i + 1, w)
            word_seq.insert(i + 2, '\"')
        i += 2
    return word_seq

In [80]:
def get_sequence_from_text(text):
    seq = split_sentence(text)
    seq = split_quotes(seq)
    return seq

In [81]:
def get_sequence_df(index, row):
    seq = get_sequence_from_text(row['text'])
    seq_df = pd.DataFrame(columns=ner_column_names)
    w_df = pd.DataFrame([['', '', '', '']], columns=ner_column_names)
    for i in range(len(seq)):
        w = seq[i]
        label = 'O'
        for lbl in labels:
            lbl_split = row[lbl].split()
            if len(w) > 0 and w in lbl_split:
                if w == lbl_split[0]:
                    label = 'B-' + lbl
                elif w_df['label'][0] == 'B-' + lbl or w_df['label'][0] == 'I-' + lbl:
                    label = 'I-' + lbl
        w_df = pd.DataFrame([[index, i, w, label]], columns=ner_column_names)
        seq_df = pd.concat([seq_df, w_df], ignore_index=True)
    seq_df['label'] = seq_df['label'].str.upper()
    return seq_df

In [82]:
ner_df = pd.DataFrame(columns=ner_column_names)
for index, row in data.iterrows():
    use_case_df = get_sequence_df(index, row)
    ner_df = pd.concat([ner_df, use_case_df], ignore_index=True)

In [83]:
ner_df[ner_df['testcase_id'] == 55]

,testcase_id,word_num,word,label
1776,55,0,продавец,B-ACTOR
1777,55,1,пытается,O
1778,55,2,удалить,O
1779,55,3,товар,O
1780,55,4,из,O
1781,55,5,магазина,O
1782,55,6,другого,O
1783,55,7,продавца,O
1784,55,8,.,O
1785,55,9,действие,O


In [84]:
ner_df[ner_df['testcase_id'] == 312]

,testcase_id,word_num,word,label
11886,312,0,администратор,B-ACTOR
11887,312,1,базы,I-ACTOR
11888,312,2,данных,I-ACTOR
11889,312,3,изменяет,O
11890,312,4,тип,O
11891,312,5,поля,O
11892,312,6,в,O
11893,312,7,таблице,O
11894,312,8,.,O
11895,312,9,действие,O


In [85]:
ner_df.describe()

,testcase_id,word_num,word,label
count,15110,15110,15110,15110
unique,400,52,1403,6
top,380,0,.,O
freq,52,400,1201,13642


In [86]:
ner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15110 entries, 0 to 15109
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   testcase_id  15110 non-null  object
 1   word_num     15110 non-null  object
 2   word         15110 non-null  object
 3   label        15110 non-null  object
dtypes: object(4)
memory usage: 472.3+ KB


In [87]:
ner_df.to_csv("ner_labeled.csv", sep=';')
ner_df.to_excel("ner_labeled.xlsx")